# MTH6412B: Projet voyageur de commerce (Phase 3)

*Florence Tanoi Namio*: **M1913387**

*Rémi Decouvelaere*: **M2035574**

## Objectif

L'objectif de la phase 3 est d'implémenter des algorithmes plus performants que celui implémenté en phase 2, dans le but de construire un arbre de recouvrement minimal à partir d'un graphe non orienté connexe existant.

## 1. Heuristiques d'accélération de l'algorithme de Kruskal

### 1.1 Union via le rang

La première heuristique d'accélération de l'algorithme de Kruskal consiste à affecter un rang à chaque sommet, correspondant à la hauteur de ce noeud dans son arborescence : l'étape d'union de l'algorithme tiendra compte des rangs des racines des deux arborescences à unir, de manière à minimiser la longueur du chemin maximal dans la nouvelle arborescence.

Nous avons donc utilisé un dictionaire **rank** qui associe à chaque noeud son rang:
```Julia
     rank=Dict(name(node) => 0 for node in nodes(G))
```

Pour le reste, nous avons repris notre première fonction **kruskal()**, en modifiant seulement la fonction réalisant l'union (remplacée par la nouvelle fonction **union_rank!**)
```Julia
"""
Fonction renvoyant un arbre de recouvrement de coût minimal associé au
 graphe G, en utilisant l'algorithme de Kruskal
 """
function kruskal_rank(G::Graph)
     E = edges(G)
     #Tri des arêtes par poids
     sort!(E, by = x -> weight(x))
    
     parents=Dict(name(node) => name(node) for node in nodes(G))
     rank=Dict(name(node) => 0 for node in nodes(G))
     #Graphe contenant les noeuds de G, initialement sans arêtes
     G_construction=Graph("Arbre", nodes(G), Edge{Vector{Float64}}[])
     for e in E
         #Si les deux noeuds de l'arête e ne sont pas dans le même ensemble connexe
        if connex(e,parents) == false
            #On ajoute cette arête au graphe de construction
            add_edge!(G_construction,e)
            #On ajoute cette arête à la forêt d'arborescence
            union_rank!(e,parents,rank)
        end
    end
    #Le graphe de construction obtenu est un arbre de recouvrement de G
    return G_construction
end
```
Notre fonction **union_rank!()** fonctionne de la façon suivante : elle compare les rangs des racines des deux arborescences, puis la racine de plus haut rang devient le parent de la deuxième racine (de façon à ne pas augmenter la longueur du chemin le plus long dans l'arborescence). Dans le cas où les rangs des deux racines sont égaux (cela signifie que les branches les plus longues de chaque arborescence ont la même longueur), une des deux racines devient parent de l'autre, et son rang augmente donc de 1.

```Julia
"""
Fonction réalisant l'union de deux arborescences, par l'arête edge,
 et dans la forêt représentée par le dictionaire dict, en tenant compte des rangs des noeuds contenus dans rank
 """
function union_rank!(edge::Edge,dict::Dict{String,String},rank::Dict{String,Int})
    #On récupère dans node1 et node2 les deux noeuds contenus dans l'arête edge
    node1 = data(edge)[1]
    node2 = data(edge)[2]
    root1 = root(node1,dict)#racine de node1
    root2 = root(node2,dict)#racine de node2
    rank1 = rank[root1]#rang de la racine de node1
    rank2 = rank[root2]#rang de la racine de node2
    #Si les rangs des deux racines sont égaux
    if rank1 == rank2
        dict[root2] = root1 #La racine de node1 devient le parent de la racine de node2
        rank[root1] += 1 #Le rang de cette racine augmente de 1
    #Si le rang de la racine de node 1 est strictement supérieur à celui de la racine de node2
    elseif rank1 > rank2
        dict[root2] = root1 #La racine de node1 devient le parent de la racine de node2
    #Si le rang de la racine de node 1 est strictement inférieur à celui de la racine de node2
    else
        dict[root1] = root2 #La racine de node2 devient le parent de la racine de node1
    end
end
```



### 1.2 Compression des chemins

La deuxième heuristique d'accélération de l'algorithme de Kruskal, encore plus efficace, consiste à remplacer le parent de chaque noeud par sa racine : ainsi on perd moins de temps à rechercher le parent d'un noeud.

Si le parent de chaque noeud est égal à sa racine, la fonction **root()** qui recherchait la racine d'un noeud ne nous sert plus à rien: nous avons utilisé ici seulement le dictionaire **parents** qui associe à chaque noeud son parent (et donc sa racine).

Nous avons remplacé notre fonction **kruskal()** par une nouvelle fonction **kruskal_compression()**, qui diffère de deux façons:
- Elle n'utilise plus la fonction **root()**
- L'union entre deux arborescences est réalisée par une nouvelle fonction **union_compression!()**

```Julia
"""
Fonction renvoyant un arbre de recouvrement de coût minimal associé au
 graphe G, en utilisant l'algorithme de Kruskal
 """
function kruskal_compression(G::Graph)
     E = edges(G)
     #Tri des arêtes par poids
     sort!(E, by = x -> weight(x))

     parents=Dict(name(node) => name(node) for node in nodes(G))
     #Graphe contenant les noeuds de G, initialement sans arêtes
     G_construction=Graph("Arbre", nodes(G), Edge{Vector{Float64}}[])
     for e in E
         #Si les deux noeuds de l'arête e ne sont pas dans le même ensemble connexe
        if connex(e,parents)==false
            #On ajoute cette arête au graphe de construction
            add_edge!(G_construction,e)
            #On ajoute cette arête à la forêt d'arborescence
            union_compression!(e,parents)
        end
    end
    #Le graphe de construction obtenu est un arbre de recouvrement de G
    return G_construction
end```
    
Voici comment nous avons réalisé l'union entre deux arborescences : nous récupérons tout d'abord la racine (c'est-à-dire le parent) de chacun des deux noeuds de l'arête. Puis, pour tous les noeuds dont le parent est la deuxième racine (y compris elle-même), nous avons remplacé leur parent par la première racine. Ainsi, dans la nouvelle arborescence, tous les noeuds ont pour parent la première racine.

```Julia
"""
Fonction réalisant l'union de deux arborescences, par l'arête edge,
 et dans la forêt représentée par le dictionaire dict
 """
function union_compression!(edge::Edge,dict::Dict{String,String})
    #On récupère dans node1 et node2 les deux noeuds contenus dans l'arête edge
    node1=data(edge)[1]
    node2=data(edge)[2]
    root1=dict[name(node1)]
    root2=dict[name(node2)]
    for (key,val) in dict #On parcourt tous les couples (noeud, parent du noeud) du dictionaire parents
        if val==root2 #Si le parent du noeud en question est root2
            dict[key]=root1 #Son parent devient root1
        end
    end
    #Ainsi tous les noeuds de l'arborescence de racine root2 ont maintenant pour parent root1
end```

### 1.3 Nouvelle fonction **kruskal**

La fonction kruskal_compression() est environ dix fois plus rapide que la fonction kruskal_rank(), elle-même dix fois plus rapide que notre fonction kruskal() originale. 

Nous avons créé une nouvelle fonction kruskal() qui utilise les deux heuristiques d'accération en même temps. Le temps d'exécution est divisé par environ 1000, par rapport à la fonction kruskal() originale.

### 1.4 Calculs sur les rangs

#### Le rang d'un noeud est toujours inférieur à |S| - 1

En effet, le rang est égal à la hauteur de la plus haute feuille dans l'arbre dont le noeud est racine. Dans le pire des cas, la branche passe une fois par tous les noeuds du graphe : la hauteur est alors |S| - 1. Le rang d'un noeud ne peut donc pas excéder |S| - 1.

#### Le rang d'un noeud est toujours inférieur à |log2(|S|)| 

En effet, essayons de construire un arbre dans lequel le rang d'un noeud n1 est maximal:
- Initialement, les rangs de tous les noeuds sont nuls
- A chaque fois que l'on souhaite augmenter de 1 le rang du noeud, il faut réaliser l'union entre ce noeud et un noeud de même rang. Or, pour que le deuxième noeud ait le même rang que le premier, il faut que l'arbre dont il est la racine ait au moins autant d'arêtes que l'arbre dont le premier noeud est racine (puisqu'on construit le premier arbre de manière à maximiser le rang de la racine avec le minimum d'arêtes)
- Ainsi, à chaque fois que l'on souhaite augmenter le rang de noeud de 1, il faut doubler le nombre d'arêtes de l'arbre.
De cette façon, si *n* est le nombre d'arêtes utilisées à une certaine étape et *r* le rang d'un des noeuds, on a toujours : n >= 2^(r-1)

Donc: log2(n) >= r - 1

Or, l'arbre de recouvrement final a exactement autant d'arêtes que de noeuds dans le graphe (|S|).

D'où: log2(|S|) >= r - 1

Donc: le rang d'un noeud est toujours inférieur à log2(|S|)-1 donc à log2(|S|)

## 2. Implémentation de l'algorithme de Prim

### Initialisations

Durant l'exécution de l'algorithme de Prim, on construit progressivement un arbre de recouvrement minimal, en partant d'un graphe *tree* constitué d'un seul sommet *root*, auquel on va ajouter à chaque itération l'arête de coût minimal reliant un des noeuds de l'arbre *tree* à un des noeuds non inclus dans l'arbre (regroupés dans la file de priorité *file*), ainsi que le noeud de l'arête qui n'est pas déjà dans l'arbre (*node2*).
```Julia
tree=Graph("Arbre",[root],Edge{T}[]) #Initialisation du futur arbre de recouvrement *tree*
```
La file de priorité *file* contient donc au départ tous les sommets du graphe principal G , à l'exception de la racine de l'arbre *root* (celle-ci peut être choisie par l'utilisateur ou laissée au hasard).
Au départ, chaque noeud, en dehors de *root* a une priorité (ou poids) infinie (c'est-à-dire, dans notre implémentation de file de priorité **PriorityQueue**, que tout noeud de priorité inférieure à l'infini sort avant ces noeuds de la file).
```Julia
file=PriorityQueue(PriorityItem[]) #File de priorité contenant tous les noeuds restants à ajouter à l'arbre tree
#Remplissage de file
for node in nodes(G)
    if node!=root
        push!(file,PriorityItem(Inf,node)) #Tous les noeuds de la file sont initialement à distance infinie de la racine
    end
end
```

On utilise également un dictionaire *weights*, qui permet d'accéder rapidement au poids d'une arête, à partir du nom de ses deux noeuds. En effet, il est autrement assez compliqué de retrouver une arête à partir du nom de ces noeuds.

```Julia
#Création d'un dictionaire associant à chaque couple de noeuds reliés par une arête le poids de cette arête
     weights=Dict{Array{String,1},Real}()
     for edge in edges(G)
         merge!(weights,Dict([name(data(edge)[1]),name(data(edge)[2])]=>weight(edge)))
         merge!(weights,Dict([name(data(edge)[2]),name(data(edge)[1])]=>weight(edge)))
     end
```

On a également choisi de créer un dictionaire **parents** qui attribue à chaque noeud du graphe son parent possible (c'est-à-dire le noeud de l'arbre le plus proche de ce noeud lorsqu'il est en-dehors de l'arbre), avant qu'il rentre dans l'arbre, qui deviendra éventuellement son parent définitif dans l'arbre. 
Ce dictionaire pourra permettre d'identifier facilement, lorsqu'on ajoute une nouvelle arête à l'arbre, le noeud auquel elle est relié 

```Julia
parents = Dict(node => node for node in nodes(G))
```

### Algorithme

Le fonctionnement de l'algorithme, tel que nous l'avons implémenté, est le suivant:
- A chaque nouvelle étape, on parcourt tous les noeuds déjà dans l'arbre et tous les noeuds pas encore dans l'arbre (c'est-à-dire dans la file de priorité). Pour chaque couple de noeuds, on regarde s'il existe une arête les reliant. 
- Si c'est le cas, et lorsque le poids de cette arête est inférieur à la priorité actuelle du noeud, on modifie celle-ci, ainsi que le parent potentiel du noeud (enregistré dans *parents*)
- Une fois qu'on a parcouru tous les couples de noeuds, on retire de la file celui qui a la plus petite 'priorité' et on l'ajoute à l'arbre *tree* ainsi que l'arête le reliant à *tree* (que l'on connaît grâce à *parents*)
- On poursuit l'algorithme jusqu'à ce que la file de priorité ne contienne plus aucun noeud


```Julia
while !is_empty(file) #tant que la file de priorité contient encore des noeuds
          for node1 in nodes(tree) #On parcourt tous les noeuds de l'arbre de construction
              for i in 1 : length(file) #On parcourt tous les noeuds qui ne sont pas encore dans l'arbre
                  node2 = data(items(file)[i])
                  weight2=priority(items(file)[i])
                  if [name(node1), name(node2)] in keys(weights) #Si l'arête reliant ces deux noeuds existe
                      if weights[[name(node1),name(node2)]] < weight2 #et si son poids est inférieur à la priorité du noeud node2 de la file
                          priority!(items(file)[i], weights[[name(node1),name(node2)]]) #On modifie la priorité du noeud
                          parents[node2] = node1 #On modifie le parent potentiel du noeud
                      end
                  end

              end
          end
          item=popfirst!(file) #L'item de plus haute priorité est retiré de la file
          node=data(item) #On retient le noeud contenu dans cet item
          add_node!(tree, node) #Il est ensuite ajouté à l'arbre
          add_edge!(tree, Edge((node, parents[node]), priority(item))) #On ajoute à l'arbre l'arête  reliant ce noeud à l'arbre
    end```
    
La fonction renvoie finalement l'arbre *tree*, de poids minimal.

### Amélioration

Le temps d'exécution de **prim()** étant supérieur à celui de **kruskal()**, nous avons cherché à améliorer **prim()**.

- Plutôt que de parcourir tous les noeuds de l'arbre à chaque fois, on met à jour seulement les noeuds qui sont reliés au noeud node1 qu'on a ajouté à l'arbre à l'étape précédente:
```Julia
node1=root
while !is_empty(file) #tant que la file de priorité contient encore des noeuds
          
          for i in 1 : length(file) #On parcourt tous les noeuds qui ne sont pas encore dans l'arbre
              node2 = data(items(file)[i])
              weight2=priority(items(file)[i])
              if [name(node1), name(node2)] in keys(weights) #Si l'arête reliant ces deux noeuds existe
                  if weights[[name(node1),name(node2)]] < weight2 #et si son poids est inférieur à la priorité du noeud node2 de la file
                      priority!(items(file)[i], weights[[name(node1),name(node2)]]) #On modifie la priorité du noeud
                      parents[node2] = node1 #On modifie le parent potentiel du noeud
                  end
              end
          end
          
          item=popfirst!(file) #L'item de plus haute priorité est retiré de la file
          node=data(item) #On retient le noeud contenu dans cet item
          add_node!(tree, node) #Il est ensuite ajouté à l'arbre
          add_edge!(tree, Edge((node, parents[node]), priority(item))) #On ajoute à l'arbre l'arête  reliant ce noeud à l'arbre
          node1=node
    end```
    
- Lorsque le graphe est complet, on a pas besoin de tester si l'arête appartient bien au graphe

## 3. Application

Nous avons testé nos fonctions **kruskal()** et **prim()** sur diverses instances de *tsp*.

Les deux fonctions renvoient des arbres de même poids, mais la fonction **kuskal()** reste environ dix fois plus rapide que **prim()**.

Nos résultats restent globalement satisfaisants car nous avons considérablement réduit le temps d'exécution de nos fonctions, par rapport à l'ancienne fonction **kruskal()**.